# Part 7

Univeral code that we will use for the entire notebook.

In [ ]:
from openai import OpenAI

# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [4]:

# Create an assistant.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    instructions="You are a helpful assistant.",
    name="Run Tester Assistant",
    metadata={
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_NrEdtjtl0i7boekETI16xpMQ', created_at=1715530621, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4-turbo', name='Run Tester Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Run Tester Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [7]:
# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread)


Thread(id='thread_Uvl90I7QlfwrV0IfVrbhQv91', created_at=1715533516, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [8]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="What is a penguin?",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_WOdQPdL0HjmDXgc6C7wwcMA9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715533526, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Uvl90I7QlfwrV0IfVrbhQv91')


msg_WOdQPdL0HjmDXgc6C7wwcMA9
[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')]
What is a penguin?
user


## Creating Runs
Runs are the engine that makes things happen with the LLM. 

In [9]:

run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

print(run)


Run(id='run_JDa8jztYhghywzCG4VLrloQQ', assistant_id='asst_NrEdtjtl0i7boekETI16xpMQ', cancelled_at=None, completed_at=None, created_at=1715533567, expires_at=1715534167, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Uvl90I7QlfwrV0IfVrbhQv91', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


### Getting the Run Status
You can't just keep looking at the run result to get the current status

In [26]:
print(run)
print("\n")
print(run.status)

Run(id='run_JDa8jztYhghywzCG4VLrloQQ', assistant_id='asst_NrEdtjtl0i7boekETI16xpMQ', cancelled_at=None, completed_at=None, created_at=1715533567, expires_at=1715534167, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Uvl90I7QlfwrV0IfVrbhQv91', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


queued


You have to get the run status by retrieving the run manually

In [22]:
runstatus = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

print(runstatus)
print("\n")
print(runstatus.id)
print(runstatus.last_error)
print(runstatus.status)

Run(id='run_JDa8jztYhghywzCG4VLrloQQ', assistant_id='asst_NrEdtjtl0i7boekETI16xpMQ', cancelled_at=None, completed_at=1715533588, created_at=1715533567, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=1715533567, status='completed', thread_id='thread_Uvl90I7QlfwrV0IfVrbhQv91', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=403, prompt_tokens=36, total_tokens=439), temperature=1.0, top_p=1.0, tool_resources={})


run_JDa8jztYhghywzCG4VLrloQQ
None
completed


Or you can poll the status manually with a loop

In [23]:
import openai
import time

another_run = client.beta.threads.runs.create(
  assistant_id=assistant.id,
  thread_id=thread.id,
)

# Retrieve initial run status
run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Poll the status while it's still queued or in progress
while run_status.status in ["queued", "in_progress"]:
    # Log current status
    print(f"Run status: {run_status.status}")
    
    # Wait for 1 second
    time.sleep(1)
    
    # Retrieve the status again
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=another_run.id)

# Optionally, print the final status or any other detail
print("Final status:", run_status.status)


Run status: queued
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Final status: completed


The best option is to save yourself some code and just use the "create_and_poll" method to do the same thing. 

When interacting with the API some actions such as starting a Run and adding files to vector stores are asynchronous and take time to complete. The SDK includes helper functions which will poll the status until it reaches a terminal state and then return the resulting object. If an API method results in an action which could benefit from polling there will be a corresponding version of the method ending in '_and_poll'.

In [27]:
auto_run_and_poll = client.beta.threads.runs.create_and_poll(
    assistant_id=assistant.id,
    thread_id=thread.id,
)

print(auto_run_and_poll)
print("\n")
print(auto_run_and_poll.id)
print(auto_run_and_poll.status)

Run(id='run_oTUOQy94rGjal8hk2m91qOaK', assistant_id='asst_NrEdtjtl0i7boekETI16xpMQ', cancelled_at=None, completed_at=1715537937, created_at=1715537921, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=1715537921, status='completed', thread_id='thread_Uvl90I7QlfwrV0IfVrbhQv91', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=351, prompt_tokens=1382, total_tokens=1733), temperature=1.0, top_p=1.0, tool_resources={})


run_oTUOQy94rGjal8hk2m91qOaK
completed
